In [186]:
using CSV

raw = CSV.read("courses_and_GPA.csv")
(m,n) = size(raw)

courses = raw[:,1]
gpa = convert(Matrix{Float64}, raw[:,2:n])
classnum = length(gpa)
pre = [[0],[0],[0],[2],[2,3],[3],[3],[69,70,3],[51],[51],[49],[51],[6,7],[55,3],[51,59,3],[70,6],
    [3,51],[51],[18,3],[69,1],[45,51],[3,69],[5,6],[5,6],[5,6],[1],[3,68],[23],[47,3,5],[4,5],
    [6,47],[69,6],[5,6],[55,73],[1],[3,69],[70,6],[0],[3,51],[0],[24],[24],[32],[0],[44],[44],
    [69],[47],[71,73],[69],[69],[58],[52],[47],[69],[47],[0],[57],[47],[47],[51],[47],[53],[56,57],
    [64],[64],[56],[0],[69],[0],[47],[48],[68]]                                 

# create P (pre-requisite matrix as in Write Up)
# so that can use P*x to simplify stuff
P = zeros(Int8, classnum, classnum)
for i in 1:size(pre,1)
    for j in 1:size(pre,1)
        if i == j  # diagnal
            # diagnal = 1 if i has no pre-req
            if pre[i][1] == 0
                P[i,j] = 1
            # diagnal = 0 if i has pre-req
            else 
                P[i,j] = 0
            end
        # if non diagonal loop through prereqs
        elseif pre[i][1] == 0 # assign 0 if i has no prereq
            P[i,j] = 0
        else 
            # loop through pre-req of i see if j is a pre-req
            for k in 1:size(pre[i],1) 
                # if class j is pre-req of i, assign 1,else assign 0
                if j == pre[i][k]
                    P[i,j] = 1
                else
                    P[i,j] = 0
                end
            end
        end
    end
end
           
P # check P 

73×73 Array{Int8,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     

In [187]:
# conversion to binary arrays as defined 

basic_cs0 = [2,3,5,6,70]
basic_math0 = [68,69]
add_math0 = [55,73,8,10,14,15,18,19,46,47,50,51,52,53,54,56,58,60,61,62,12,63,64,65,66,67,48,49,71,72]
theory0 = [37,16]
soft_hard0 = [7,13,23,24,30,33,41,42]
app0 = [8,9,14,15,17,18,22,27,28,29,32,35,]
elect0 = [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,40]
len = length(gpa);


# function converts above kind of array into ones like in the final project write up math model
# so that multiplication are easier 
function array_to_bin(arraytoconvert)
    converted=zeros(classnum)
    for i in 1:classnum
        for j in 1:size(arraytoconvert,1)
            if i == arraytoconvert[j]
                converted[i] = 1
            end
        end
    end
    return converted
end


# converts all above data
basic_cs = array_to_bin(basic_cs0)
basic_math = array_to_bin(basic_math0)
add_math =array_to_bin(add_math0)
theory = array_to_bin(theory0)
soft_hard = array_to_bin(soft_hard0)
app = array_to_bin(app0)
elect = array_to_bin(elect0)


73-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [198]:
# integer params to change
TOT_COURSES_UP = 40  # total courses upper bound
TOT_COURSES_LO = 30  # total courses lower bound
SEM_COURSES_UP = 5  # semester courses upper bound
SEM_COURSES_LO = 1  # semester courses lower bound
NUM_SEMESTER = 10  # int (max semesters)


# Every requirement list must x8 (or the semester number)
# instantiate for concact
basic_cs1 = Float64[]
basic_math1 = Float64[]
add_math1 = Float64[]
theory1 = Float64[]
soft_hard1 = Float64[]
app1 = Float64[]
elect1 = Float64[]
gpa1 = Float64[] # need to edit GPA as it will be easier to multiply with x

# # concact vectors 
for i in 1:NUM_SEMESTER
    print("here")
    basic_cs1 = vcat(basic_cs1, basic_cs)
    basic_math1 = vcat(basic_math1, basic_math)
    add_math1 = vcat(add_math1, add_math)
    theory1 = vcat(theory1, theory)
    soft_hard1 = vcat(soft_hard1, soft_hard)
    app1 = vcat(app1, app)
    elect1 = vcat(elect1, elect)
    gpa1 = vcat(gpa1, gpa)
end
gpa1

herehereherehereherehere

438×1 Array{Float64,2}:
 2.844228095
 3.260386683
 3.183544304
 3.006989892
 3.069396492
 3.3        
 3.632332879
 3.003965631
 3.220930233
 2.880407125
 3.525      
 3.099164927
 3.452475248
 ⋮          
 3.234076433
 3.011659118
 2.888764464
 3.307560137
 3.113636364
 2.976923077
 2.849711981
 2.766288758
 3.027873353
 3.155594069
 3.337140796
 3.157733538

In [199]:
# using Gurobi here
using JuMP, Cbc, Gurobi, LinearAlgebra

# m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0)) Cdc didn't work with MathOptInterface.ScalarQuadraticFunction

m = Model(with_optimizer(Gurobi.Optimizer, NumericFocus=3,BarHomogeneous=1))

@variable(m, x[1:len*NUM_SEMESTER], Bin)  # create a 73 * 8 (semester number) course list

# basic catogorical course requirement
@constraint(m, basic_cs1'*x >= 5)
@constraint(m, basic_math1'*x >= 2)
@constraint(m, add_math1'*x >= 2)
@constraint(m, theory1'*x >= 1)
@constraint(m, soft_hard1'*x >= 2)
@constraint(m, app1'*x >= 1)
@constraint(m, elect1'*x >= 2)


# semester course load preference (except for 1st semester)
@constraint(m, sum(x[1:73]) <= SEM_COURSES_UP) # upper bound
@constraint(m, sum(x[1:73]) >= SEM_COURSES_LO) # lower bound

taken = x[1:73]  # keeping track of a list of taken courses to avoid repeats
satisfied= diag(P)  # keeping track of a list of satisfied courses for pre-req requirement

for i in 1:(NUM_SEMESTER-1)
    # A class cannot count for more than one degree requirement (no repeats)
    @constraint(m, taken' * x[(73*(i)+1):73*(i+1)] == 0)
    taken = taken + x[(73*(i)+1):73*(i+1)] # update list
    # Must satisfy prerequisite requirements
    @constraint(m, satisfied .- x[(73*(i-1)+1):73*i] .>= 0)
    satisfied = satisfied + P*x[(73*(i-1)+1):73*i] # update list
    # semester course load preference (except for 1st semester)
    @constraint(m, sum(x[(73*(i)+1):73*(i+1)]) <= SEM_COURSES_UP) # upper bound
    @constraint(m, sum(x[(73*(i)+1):73*(i+1)]) >= SEM_COURSES_LO) # lower bound
end


# courses total preference 
@constraint(m, sum(x[i] for i in 1:len*NUM_SEMESTER) <= TOT_COURSES_UP) # upper bound
@constraint(m, sum(x[i] for i in 1:len*NUM_SEMESTER) >= TOT_COURSES_LO) # lower bound

# starting course preference
@constraint(m, x[2] == 1)
@constraint(m, x[3] == 1)

# Maximize GPA
@objective(m, Max, (gpa1'*x)[1]);


print(m)
optimize!(m)



Academic license - for non-commercial use only
Max 2.844228095 x[1] + 3.260386683 x[2] + 3.183544304 x[3] + 3.006989892 x[4] + 3.069396492 x[5] + 3.3 x[6] + 3.632332879 x[7] + 3.003965631 x[8] + 3.220930233 x[9] + 2.880407125 x[10] + 3.525 x[11] + 3.099164927 x[12] + 3.452475248 x[13] + 3.050802139 x[14] + 3.143712575 x[15] + 3.003384095 x[16] + 3.306451613 x[17] + 2.949211909 x[18] + 2.6875 x[19] + 3.408959538 x[20] + 3.427698574 x[21] + 3.285 x[22] + 3.165486726 x[23] + 3.372149424 x[24] + 3.23 x[25] + 3.105 x[26] + 3.137957856 x[27] + 3.776315789 x[28] + 3.58 x[29] + 3.134720701 x[30] + 3.655172414 x[31] + 3.108283433 x[32] + 3.401515152 x[33] + 3.147727273 x[34] + 3.60864745 x[35] + 3.274617068 x[36] + 3.149271002 x[37] + 3.862068966 x[38] + 3.51147541 x[39] + 3.38 x[40] + 3.264409881 x[41] + 3.386850153 x[42] + 3.726851852 x[43] + 3.10864486 x[44] + 3.022779795 x[45] + 2.917865707 x[46] + 2.87695035 x[47] + 3.026936317 x[48] + 3.04313205 x[49] + 2.939852269 x[50] + 2.812000537 x[5

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 388 rows, 438 columns and 3415 nonzeros
Model fingerprint: 0x1dc5d82d
Model has 5 quadratic constraints
Variable types: 0 continuous, 438 integer (438 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 230 rows and 207 columns
Presolve time: 0.02s
Presolved: 158 rows, 231 columns, 1633 nonzeros
Variable types: 0 continuous, 231 integer (231 binary)

Root relaxation: objective 1.006905e+02, 123 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  100.69051    0   28          -  100.69051      -     -    0s
H    0     0                      99.9918390  100.69051  0.

In [200]:
# printing out final results. all the strange +1 -1 is because Julia starts at 1
opt_gpa = 0
for i in 1:len*NUM_SEMESTER
    n = i%len
    if n == 0  
        n = len # because Julia starts at 1
    elseif n == 1 # because Julia starts at 1
        println("=======================")
        print("Starting Semester ")
        println(trunc(Int,(i-1)/len+1))
        println("=======================")
    end
    if (value(x[i]) > 0)
        print(courses[n])
        print(" - ")
        println(round(gpa[n],digits=1))
        opt_gpa += gpa[n]
    end
end


print("Overall Predicted GPA:")
println((gpa1'*JuMP.value.(x))/sum(JuMP.value.(x)))

Starting Semester 1
COMP SCI 200 - 2.8
COMP SCI 252 - 3.3
COMP SCI 300 - 3.2
MATH 375 - 3.3
MATH 221 - 2.8
Starting Semester 2
COMP SCI 354 - 3.1
COMP SCI 400 - 3.3
COMP SCI 407 - 3.6
COMP SCI 570 - 3.6
MATH 376 - 3.3
Starting Semester 3
COMP SCI 536 - 3.2
COMP SCI 537 - 3.4
COMP SCI 547 - 3.6
COMP SCI 559 - 3.1
COMP SCI 579 - 3.9
Starting Semester 4
COMP SCI 506 - 3.5
COMP SCI 545 - 3.8
COMP SCI 564 - 3.4
COMP SCI 642 - 3.4
COMP SCI 679 - 3.7
Starting Semester 5
COMP SCI 532 - 3.4
COMP SCI 577 - 3.1
COMP SCI 639 - 3.4
COMP SCI 640 - 3.3
MATH 240 - 3.0
Starting Semester 6
COMP SCI 471 - 3.5
COMP SCI 558 - 3.7
COMP SCI 635 - 3.5
MATH 443 - 3.4
MATH 222 - 2.8
Overall Predicted GPA:[3.3414284512]


In [160]:
JuMP.value(x[1])
classnum

73